<a href="https://colab.research.google.com/github/AngelOD565/youtube_api_data_extraction/blob/main/youtubeAPI_to_RDS_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **API Call to Youtube for Data Extraction**

In [ ]:
#import libraries
import requests
import pandas as pd
import time

In [ ]:
#Keys
API_Key = "x"
Channel_ID = "UCqffvr3tqWOR5Oyq16t5-kQ"

In [ ]:
#make API call
pageToken = ""
url = "https://www.googleapis.com/youtube/v3/search?key="+API_Key+"&channelId="+Channel_ID+"&part=snippet,id&order=date&maxResults=10000"+pageToken

response = requests.get(url).json()

In [ ]:
#create function to get and return video statistics
def get_video_details(video_id):    

    #API call collecting view, like, and comment counts
    url_video_stats = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+"&part=statistics&key="+API_Key
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    comment_count= response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, comment_count

In [ ]:
#create function to get video details and append them into our dataframe
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_Key+"&channelId="+Channel_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df
  

In [ ]:
#main

#build our dataframe
df = pd.DataFrame(columns=['video_id','video_title','upload_date','view_count','like_count','comment_count'])

df = get_videos(df)

In [ ]:
df2

In [ ]:
#create second dataframe removing duplicates from API call
df2 = df.drop_duplicates(keep='first')

# **Uploading dataframe to AWS RDS Database Instance**

In [ ]:
#install package and import library
!pip install psycopg2
import psycopg2 as ps

In [ ]:
#define function to connect to our database
def connect_to_db(host_name, dbname, port, username, password):
  try:
    conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

  except ps.OperationalError as e:
    raise e
  else:
    print('Connected!')
  return conn


In [ ]:
#create table
def create_table(curr):
  create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
    video_id VARCHAR(255) PRIMARY KEY,
    Video_title TEXT NOT NULL,
    upload_date DATE  NOT NULL DEFAULT CURRENT_DATE,
    view_count INTEGER NOT NULL,
    like_count INTEGER NOT NULL,
    comment_count INTEGER NOT NULL
  )""")
  curr.execute(create_table_command)

In [ ]:
#insert command

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
      insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                          view_count, like_count, comment_count)
      VALUES(%s,%s,%s,%s,%s,%s);""")
      row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
      curr.execute(insert_into_videos, row_to_insert)

In [ ]:
#update row if video exists
def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
  query = ("""UPDATE videos
              SET video_title = %s,
                  view_count = %s,
                  like_count = %s,
                  comment_count = %s
              WHERE video_id = %s;""")

  vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
  curr.execute(query, vars_to_update)


In [ ]:
#check if video exists
def check_if_video_exists(curr, video_id):
  query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
  curr.execute(query, (video_id,))

  return curr.fetchone() is not None


In [ ]:
def append_from_df2_to_db(curr,df2):
    for i, row in df2.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])

In [ ]:
#update database
#handling data for scalability

#check to see if video already exists

def update_db(curr,df2):
    tmp_df2 = pd.DataFrame(columns=['video_id','video_title','upload_date','view_count','like_count','comment_count']) #create temp df2 as list of new items to append to database later

    for i, row in df2.iterrows():
      if check_if_video_exists(curr, row['video_id']): #if video already exists, then update features
        update_row(curr, row['video_id'], row['video_title'], row['view_count'], row['like_count'], row['comment_count'])
      else: #if video doesn't exist, we append it to the db
        tmp_df2 = tmp_df2.append(row)
    return tmp_df2


In [ ]:
#Main

#configure parameters to our specific database then connect
host_name = 'database-yt.cgw9v1phodos.us-east-1.rds.amazonaws.com'
dbname = ''
port = '5432'
username = 'postgres'
password = 'postgres'

#establish connection
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

In [ ]:
#create table
create_table(curr)

In [ ]:
new_vids_df2 = update_db(curr, df2)

In [ ]:
append_from_df2_to_db(curr, new_vids_df2)

In [ ]:
conn.commit()